<a href="https://colab.research.google.com/github/azadranjith/kaggle-competitions/blob/main/plant_seedling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## kaggle api 

In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p  ~/.kaggle
!cp kaggle.json ~/.kaggle

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!pwd

/content


In [ ]:

!kaggle datasets list

In [ ]:
!kaggle competitions download -c plant-seedlings-classification

100% 1.68G/1.69G [00:14<00:00, 133MB/s]
100% 1.69G/1.69G [00:14<00:00, 129MB/s]


In [ ]:
!unzip plant-seedlings-classification.zip

# **trained model using fastai library**

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

Mounted at /content/gdrive


In [ ]:
from fastbook import *
from fastai.vision.all import *

In [ ]:
path = Path('/content/data/train')

In [ ]:
path

Path('/content/data/train')

In [ ]:
def parent_folder(x):
  try:
    pattern = '\/([\dA-Za-z -]*)\/[A-Z_a-z\d]+.png'
    match = re.findall(pattern,str(x))
    for i in match:
      if i != '':
        cat = i
      return cat
  except:
    print('check these files')
    print(x)


# DATALOADERS FROM DATA BLOCK API

In [ ]:
plant = DataBlock(
    blocks = (ImageBlock, CategoryBlock),
    get_items = get_image_files,
    get_y = parent_folder,
    splitter = RandomSplitter(valid_pct=0.2,seed = 34),
    item_tfms=Resize(126))

In [ ]:
dls = plant.dataloaders(path)

In [ ]:
dls.train.show_batch(max_n=9)

In [ ]:
learner = vision_learner(dls,resnet34,metrics = accuracy)
learner.fine_tune(4)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:136: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and will be removed in 0.15. Please use keyword parameter(s) instead.
  f"Using {sequence_to_str(tuple(keyword_only_kwargs.keys()), separate_last='and ')} as positional "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,accuracy,time
0,2.490072,1.323253,0.594737,07:15


epoch,train_loss,valid_loss,accuracy,time
0,1.095345,0.742552,0.752632,10:16
1,0.549149,0.549057,0.831579,10:16
2,0.256951,0.521970,0.837895,10:15
3,0.128562,0.508429,0.848421,11:03


# TESTING

In [ ]:
learner.export('plant_seedling01.pkl')

In [ ]:
test_path = get_image_files('/content/data/test')

In [ ]:
sugar_beet = get_image_files('/content/data/train/Sugar beet')
pred = learner.predict(sugar_beet[34])
pred

('Sugar beet',
 TensorBase(11),
 TensorBase([1.7580e-05, 2.9635e-05, 1.3889e-07, 1.0752e-05, 9.0298e-05, 1.6854e-02, 9.9054e-07, 2.9431e-04, 1.9004e-07, 3.7310e-06, 5.8318e-06, 9.8269e-01]))

('Black-grass',
 TensorBase(0),
 TensorBase([9.9857e-01, 2.2546e-06, 9.2124e-07, 2.4200e-06, 1.3824e-05, 2.2384e-06, 1.3452e-03, 3.7050e-06, 2.1453e-05, 8.3034e-06, 2.4335e-05, 7.1337e-07]))

In [ ]:
prediction = learner.predict(test_path[0])
prediction

('Loose Silky-bent',
 TensorBase(6),
 TensorBase([2.9587e-03, 2.9654e-05, 2.4491e-03, 4.8999e-04, 8.1264e-05, 9.1195e-02, 9.0207e-01, 5.9485e-05, 1.9862e-05, 1.2765e-04, 5.8011e-05, 4.6435e-04]))

In [ ]:
prediction[0]

'Loose Silky-bent'

In [ ]:
dls.vocab

['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent', 'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet', 'test']

# WRITING TO CSV FOR SUBMISSION

In [ ]:
import csv

In [ ]:
#to get file name
def image_name(x):
  try:
    pattern='\/[\dA-Za-z -]*\/([A-Z_a-z\d]+.png)'
    match = re.findall(pattern,str(x))
    for i in match:
      if match != '':
        cat = i
    return cat
  except:
    print('check these files')
    print(x)


In [ ]:
#get images from test folder
path = Path('/content/data/test')
fnames = get_image_files(path)
fnames

(#794) [Path('/content/data/test/3d38a87bc.png'),Path('/content/data/test/f4234cf4f.png'),Path('/content/data/test/3f92d8039.png'),Path('/content/data/test/35ebe165c.png'),Path('/content/data/test/0ebf8f2f4.png'),Path('/content/data/test/39d569be4.png'),Path('/content/data/test/85431c075.png'),Path('/content/data/test/55920f07f.png'),Path('/content/data/test/bf3924a57.png'),Path('/content/data/test/b47691c08.png')...]

In [ ]:
rows = []
j = 0
for i in fnames:
  
  x = image_name(i)
  pred = learner.predict(fnames[j])
  row = [x,pred[0]]
  rows.append(row)
  j += 1
print(rows)

[['3d38a87bc.png', 'Loose Silky-bent'], ['f4234cf4f.png', 'Fat Hen'], ['3f92d8039.png', 'Black-grass'], ['35ebe165c.png', 'Loose Silky-bent'], ['0ebf8f2f4.png', 'Charlock'], ['39d569be4.png', 'Loose Silky-bent'], ['85431c075.png', 'Loose Silky-bent'], ['55920f07f.png', 'Maize'], ['bf3924a57.png', 'Scentless Mayweed'], ['b47691c08.png', 'Scentless Mayweed'], ['506347cfe.png', 'Cleavers'], ['b29339405.png', 'Loose Silky-bent'], ['eec1079a1.png', 'Charlock'], ['1364b297a.png', 'Cleavers'], ['aecfaed64.png', 'Shepherds Purse'], ['5f04aed97.png', 'Small-flowered Cranesbill'], ['6db684fff.png', 'Scentless Mayweed'], ['632156793.png', 'Cleavers'], ['4b032563b.png', 'Common Chickweed'], ['32c86784b.png', 'Common Chickweed'], ['bb7621cb3.png', 'Fat Hen'], ['8e2e5604e.png', 'Sugar beet'], ['2ff5cb348.png', 'Sugar beet'], ['539961189.png', 'Common Chickweed'], ['1d48b7564.png', 'Loose Silky-bent'], ['fc6f686fb.png', 'Sugar beet'], ['30ad31220.png', 'Charlock'], ['862b8e7a0.png', 'Sugar beet'], ['

In [ ]:
len(rows)

794

In [ ]:
fields = ['file','species']
file_name = 'submission_csv.csv'
with open(file_name,'w') as csvfile:
  csvwriter = csv.writer(csvfile)

  csvwriter.writerow(fields)
  csvwriter.writerows(rows)

In [ ]:
!pwd

/content


In [ ]:
!kaggle competitions submit -c plant-seedlings-classification -f submission.csv -m "ranjith"

100% 23.1k/23.1k [00:00<00:00, 55.9kB/s]
Successfully submitted to Plant Seedlings Classification